# Data Preparation: Value Transformations with GNIS & Baseball Examples

In [1]:
import numpy as np
import pandas as pd

Only run the following if you need to recreate the baseball database:

In [3]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!psql -h localhost -d baseball -f data/baseball.sql

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
COPY 5219
COPY 104256
COPY 179
COPY 6236
COPY 425
COPY 6879
COPY 104324
COPY 13943
COPY 17350
COPY 138838
COPY 12028
COPY 31955
COPY 13110
COPY 4191
COPY 3040
COPY 3469
COPY 93
COPY 252
COPY 19370
COPY 45806
COPY 5445
COPY 26428
COPY 1207
COPY 325
COPY 2865
COPY 120
COPY 52
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE

In [2]:
## we'll use the Lahman baseball database in our examples today.
%reload_ext sql
%sql postgresql://localhost:5432/baseball
%config SqlMagic.displaylimit = 20

Connecting to 'postgresql://localhost:5432/baseball'

---
# Scalar and Window Function Demos: Baseball database
## [At Home, Slides in Class] Scalar Functions and Query Plans

Let's analyze the number of scalar functions in the following query:

In [4]:
%%sql
WITH year_num AS
  (SELECT year_id, (year_id % 100) as year
     FROM batting
  )
SELECT year_id, CONCAT('''', LPAD(year::text, 2, '0')) as year
  FROM year_num
 LIMIT 5;

Running query in 'postgresql://localhost:5432/baseball'

5 rows affected.

year_id,year
1871,'71
1871,'71
1871,'71
1871,'71
1871,'71


We can analyze it by looking at the EXPLAIN with VERBOSE (we've flattened it for convenience):

In [5]:
%%sql
EXPLAIN (VERBOSE true)
SELECT year_id,
       CONCAT('''', LPAD((year_id % 100)::text, 2, '0')) AS year
FROM batting;


Running query in 'postgresql://localhost:5432/baseball'

2 rows affected.

QUERY PLAN
Seq Scan on public.batting (cost=0.00..3927.29 rows=104324 width=36)
"Output: year_id, concat('''', lpad(((year_id % 100))::text, 2, '0'::text))"


What if scalar functions mention multiple tables?

The below query computes an arbitrary statistic for pitchers:
* 1 point for every strikeout they throw as pitcher
* –1 for every point they themselves struck out as batter

If the notebook-like output is hard to read, try out the query in `psql`. Note that notebooks don't preserve whitespace when displaying dataframes.

In [6]:
%%sql
EXPLAIN (VERBOSE true)
SELECT p.player_id, p.so - b.so
  FROM pitching p
  INNER JOIN batting b
  ON p.player_id=b.player_id;

Running query in 'postgresql://localhost:5432/baseball'

11 rows affected.

QUERY PLAN
Nested Loop (cost=0.43..12982.97 rows=341778 width=13)
"Output: p.player_id, (p.so - b.so)"
-> Seq Scan on public.pitching p (cost=0.00..1374.06 rows=45806 width=13)
"Output: p.player_id, p.year_id, p.stint, p.team_id, p.lg_id, p.w, p.l, p.g, p.gs, p.cg, p.sho, p.sv, p.ipouts, p.h, p.er, p.hr, p.bb, p.so, p.baopp, p.era, p.ibb, p.wp, p.hbp, p.bk, p.bfp, p.gf, p.r, p.sh, p.sf, p.gidp"
-> Memoize (cost=0.43..0.73 rows=7 width=13)
"Output: b.so, b.player_id"
Cache Key: p.player_id
Cache Mode: logical
-> Index Scan using batting_pkey on public.batting b (cost=0.42..0.72 rows=7 width=13)
"Output: b.so, b.player_id"


---
## Window Functions

Window functions, without partitions first, and then with partitions. <br/>


In [5]:
%%sql
SELECT name_first, name_last, year_id, HR,
       rank() OVER (ORDER BY HR DESC)
       -- , avg(HR)    OVER (PARTITION BY b.player_id ORDER BY year_id ROWS 3 PRECEDING) as avg_4yr
       -- , lag(HR, 7) OVER (PARTITION BY b.player_id ORDER BY year_id) as previous
       -- , lag(HR, 2) OVER (PARTITION BY b.player_id ORDER BY year_id) as lag2
FROM batting b, people p
WHERE p.player_id = b.player_id
   AND (name_last = 'Bonds' or name_last = 'Ruth')
ORDER BY HR DESC
LIMIT 10;

Running query in 'postgresql://localhost:5432/baseball'

10 rows affected.

name_first,name_last,year_id,hr,rank,lag1
Barry,Bonds,2001,73,1,49
Babe,Ruth,1927,60,2,47
Babe,Ruth,1921,59,3,54
Babe,Ruth,1920,54,4,29
Babe,Ruth,1928,54,4,60
Barry,Bonds,2000,49,6,34
Babe,Ruth,1930,49,6,46
Babe,Ruth,1926,47,8,25
Barry,Bonds,1993,46,9,34
Barry,Bonds,2002,46,9,73


Same query, different order by - so that we can inspect the other attributes
<br/>
Note ROWS 3 PRECEDING = RANGE BETWEEN 3 PRECEDING AND CURRENT ROW

In [11]:
%%sql
SELECT name_first, name_last, year_id, HR
       -- rank() OVER (ORDER BY HR DESC)
       , avg(HR)    OVER (PARTITION BY b.player_id ORDER BY year_id ROWS 3 PRECEDING) as avg_4yr
       , lag(HR, 1) OVER (PARTITION BY b.player_id ORDER BY year_id) as previous
       , lag(HR, 2) OVER (PARTITION BY b.player_id ORDER BY year_id) as lag2
FROM batting b, people p
WHERE p.player_id = b.player_id
   AND (name_last = 'Bonds' or name_last = 'Ruth')
ORDER BY b.player_id, year_id 
LIMIT 20;

Running query in 'postgresql://localhost:5432/baseball'

20 rows affected.

name_first,name_last,year_id,hr,avg_4yr,previous,lag2
Barry,Bonds,1986,16,16.0000000000000000,None,None
Barry,Bonds,1987,25,20.5000000000000000,16,None
Barry,Bonds,1988,24,21.6666666666666667,25,16
Barry,Bonds,1989,19,21.0000000000000000,24,25
Barry,Bonds,1990,33,25.2500000000000000,19,24
Barry,Bonds,1991,25,25.2500000000000000,33,19
Barry,Bonds,1992,34,27.7500000000000000,25,33
Barry,Bonds,1993,46,34.5000000000000000,34,25
Barry,Bonds,1994,37,35.5000000000000000,46,34
Barry,Bonds,1995,33,37.5000000000000000,37,46


---
## [At Home] Inverse Distribution Window Functions

In [12]:
%%sql
SELECT MIN(HR),
       percentile_cont(0.25) WITHIN GROUP (ORDER BY HR) AS p25,
       percentile_cont(0.50) WITHIN GROUP (ORDER BY HR) AS median,
       percentile_cont(0.75) WITHIN GROUP (ORDER BY HR) AS p75,
       percentile_cont(0.99) WITHIN GROUP (ORDER BY HR) AS p99,
       MAX(HR),
       AVG(HR) AS "average hit rate"
FROM batting;

Running query in 'postgresql://localhost:5432/baseball'

1 rows affected.

min,p25,median,p75,p99,max,average hit rate
0,0.0,0.0,2.0,31.0,73,2.8315823779763046


Now just to verify that the max is 73, we can run the following query:

In [13]:
%%sql
SELECT HR, COUNT(*) FROM batting GROUP BY HR ORDER BY HR DESC;

Running query in 'postgresql://localhost:5432/baseball'

67 rows affected.

hr,count
73,1
70,1
66,1
65,1
64,1
63,1
61,1
60,1
59,2
58,3


---
### [At Home, Slides in Class] Hypothetical-Set Window Functions

In [3]:
hrs = 70 # hypothetically, four home runs

In [4]:
%%sql
SELECT {{hrs}} as hypothetical,
       rank({{hrs}}) WITHIN GROUP (ORDER BY HR DESC),
       dense_rank({{hrs}}) WITHIN GROUP (ORDER BY HR DESC),
       percent_rank({{hrs}}) WITHIN GROUP (ORDER BY HR DESC) * 100 AS pct_rank,
       cume_dist({{hrs}}) WITHIN GROUP (ORDER BY HR)
FROM batting;

Running query in 'postgresql://localhost:5432/baseball'

1 rows affected.

hypothetical,rank,dense_rank,pct_rank,cume_dist
70,2,2,0.000958552202752962,0.9999904145698538


Without `jupysql` variable substituion

In [5]:
%%sql
SELECT 4 as hypothetical,
       rank(4) WITHIN GROUP (ORDER BY HR DESC),
       dense_rank(4) WITHIN GROUP (ORDER BY HR DESC),
       percent_rank(4) WITHIN GROUP (ORDER BY HR DESC) * 100 AS pct_rank,
       cume_dist(4) WITHIN GROUP (ORDER BY HR)
FROM batting;

Running query in 'postgresql://localhost:5432/baseball'

1 rows affected.

hypothetical,rank,dense_rank,pct_rank,cume_dist
4,18420,63,17.655573022506807,0.823445962137551


# Granularity Demos: GNIS

This notebook transforms the existing [Geographics Names Information Systems (GNIS)](https://www.usgs.gov/core-science-systems/ngp/board-on-geographic-names/download-gnis-data) national zip file.

We have provided a subset of the sql database for you in `data/national.sql`.



In [34]:
!psql -h localhost -d gnis -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database() AND pid <> pg_backend_pid();'
!psql -h localhost -c 'DROP DATABASE IF EXISTS gnis'
!psql -h localhost -c 'CREATE DATABASE gnis' 
!psql -h localhost -d gnis -f data/gnis.sql

 pg_terminate_backend 
----------------------
(0 rows)

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 3195
COPY 11533
CREATE INDEX


In [35]:
%reload_ext sql
%sql postgresql://localhost:5432/gnis
%config SqlMagic.displaylimit = 15

Switching to connection 'postgresql://localhost:5432/gnis'

* View schema in `psql`
* View some rows below

In [5]:
%sql SELECT COUNT(*) FROM national;

Running query in 'postgresql://localhost:5432/gnis'

1 rows affected.

count
11533


In [6]:
%sql SELECT * FROM national WHERE county_name = 'Alameda';

Running query in 'postgresql://localhost:5432/gnis'

21 rows affected.

feature_id,feature_name,feature_class,state_alpha,state_numeric,county_name,county_numeric,primary_lat_dms,prim_long_dms,prim_lat_dec,prim_long_dec,source_lat_dms,source_long_dms,source_lat_dec,source_long_dec,elev_in_m,elev_in_ft,map_name,date_created,date_edited
218316,Apperson Creek,Stream,CA,6,Alameda,1.0,373349N,1215000W,37.5635453,-121.8332887,373232N,1214804W,37.5422222,-121.8011111,148.0,486.0,La Costa Valley,01/19/1981,None
225998,Irvington High School,School,CA,6,Alameda,1.0,373126N,1215801W,37.523814,-121.9670659,None,None,None,None,13.0,43.0,Niles,01/19/1981,03/31/2021
226951,Laurel Elementary School,School,CA,6,Alameda,1.0,374734N,1221147W,37.792899,-122.1964288,None,None,None,None,68.0,223.0,Oakland East,06/14/2000,03/14/2021
229367,Murray Elementary School,School,CA,6,Alameda,1.0,374318N,1215557W,37.721801,-121.9326269,None,None,None,None,112.0,367.0,Dublin,01/19/1981,03/14/2021
235581,Strawberry Creek,Stream,CA,6,Alameda,1.0,375221N,1221443W,37.8724258,-122.2452464,375251N,1221354W,37.8807588,-122.2316349,154.0,505.0,Oakland East,01/19/1981,08/31/2016
1654274,Hayward Golf Course,Locale,CA,6,Alameda,1.0,373726N,1220250W,37.6238222,-122.0471843,None,None,None,None,5.0,16.0,Newark,01/19/1981,None
1664964,KOFY-AM (San Mateo),Tower,CA,6,Alameda,1.0,374934N,1221842W,37.8260385,-122.3116366,None,None,None,None,2.0,7.0,Oakland West,07/01/1994,None
1670278,Lake Elizabeth,Lake,CA,6,Alameda,1.0,373255N,1215742W,37.5487056,-121.9617554,None,None,None,None,16.0,52.0,Niles,11/09/1995,03/07/2019
1692819,California School for the Deaf - Fremont,School,CA,6,Alameda,1.0,373334N,1215747W,37.5593966,-121.9631843,None,None,None,None,20.0,66.0,Niles,05/08/1996,09/16/2016
1692863,J A Freitas Library,Building,CA,6,Alameda,1.0,374335N,1220925W,37.7263185,-122.1569101,None,None,None,None,19.0,62.0,San Leandro,05/08/1996,None


In [7]:
%%sql
SELECT *
FROM national TABLESAMPLE BERNOULLI(10);

Running query in 'postgresql://localhost:5432/gnis'

1156 rows affected.

feature_id,feature_name,feature_class,state_alpha,state_numeric,county_name,county_numeric,primary_lat_dms,prim_long_dms,prim_lat_dec,prim_long_dec,source_lat_dms,source_long_dms,source_lat_dec,source_long_dec,elev_in_m,elev_in_ft,map_name,date_created,date_edited
8884,Oak Spring,Spring,AZ,4,Mohave,15.0,364903N,1133510W,36.8174811,-113.5860693,None,None,None,None,1605.0,5266.0,Wolf Hole Mountain East,02/08/1980,None
13332,Walthenberg Rapids,Rapids,AZ,4,Coconino,5.0,361418N,1122425W,36.2383651,-112.4068345,None,None,None,None,657.0,2155.0,Explorers Monument,02/08/1980,06/07/2011
13547,West Well,Well,AZ,4,Navajo,17.0,350208N,1102857W,35.0355772,-110.4826281,None,None,None,None,1517.0,4977.0,Humpy Camp Well,02/08/1980,03/27/2018
22969,Gyp Reservoir,Reservoir,AZ,4,Mohave,15.0,363523N,1132033W,36.5896972,-113.3426134,None,None,None,None,1463.0,4800.0,Russell Spring,06/27/1984,03/16/2019
24724,Bethany Villa Adult Mobile Home Park,Populated Place,AZ,4,Maricopa,13.0,333134N,1120951W,33.5261527,-112.1640412,None,None,None,None,352.0,1155.0,Glendale,06/27/1984,None
24774,Central Ridge,Populated Place,AZ,4,Maricopa,13.0,331910N,1115312W,33.3194929,-111.8865296,None,None,None,None,364.0,1194.0,Guadalupe,06/27/1984,None
25467,Caterpillar Tank Wash,Stream,AZ,4,Maricopa,13.0,334332N,1121738W,33.725592,-112.2937718,334749N,1121933W,33.7969796,-112.3257187,384.0,1260.0,Calderwood Butte,11/01/1992,None
28394,Deer Spring Mountain,Summit,AZ,4,Navajo,17.0,340039N,1095831W,34.0107604,-109.9752047,None,None,None,None,2269.0,7444.0,Indian Pine,02/08/1980,08/16/2021
31214,Lone Pine Tank,Reservoir,AZ,4,Coconino,5.0,343219N,1113022W,34.5386834,-111.5059796,None,None,None,None,1949.0,6394.0,Buckhorn Mountain,02/08/1980,03/19/2019
31270,Lons Canyon,Valley,AZ,4,Navajo,17.0,341558N,1101409W,34.266067,-110.2358535,341621N,1101728W,34.2724081,-110.2910287,1980.0,6496.0,Pinedale,02/08/1980,03/09/2021


# [At Home] Numerical Granularity

In [10]:
%sql SELECT elev_in_m FROM National LIMIT 2;

Running query in 'postgresql://localhost:5432/gnis'

2 rows affected.

elev_in_m
931.0
2707.0


In [11]:
%%sql
SELECT elev_in_m, 
    (elev_in_m / 100)::INTEGER AS quantized,
    ((elev_in_m / 100)::INTEGER) * 100 AS round_to_100,
    SUBSTRING(elev_in_m::TEXT, 1, 2),
    CONCAT(SUBSTRING(elev_in_m::TEXT, 1, 2), '00') AS substring2
FROM National
LIMIT 5;

Running query in 'postgresql://localhost:5432/gnis'

5 rows affected.

elev_in_m,quantized,round_to_100,substring,substring2
931.0,9,900,93,9300
2707.0,27,2700,27,2700
275.0,3,300,27,2700
1685.0,17,1700,16,1600
1354.0,14,1400,13,1300


In [12]:
%config SqlMagic.named_parameters=True

/srv/conda/envs/notebook/lib/python3.11/site-packages/sql/traits.py:20: FutureWarning: named_parameters: boolean values are now deprecated. Value True will be treated as "enabled". 
Please use a valid option: "warn", "enabled", or "disabled". 
For more information, see the docs: https://jupysql.ploomber.io/en/latest/api/configuration.html#named-parameters
  warnings.warn(


In [14]:
right_shift = '>>'
left_shift = '<<'

In [15]:
%%sql
/* Since jupysql does not like bitshifts, we can fake it with string interoplation. */
SELECT elev_in_m,
    (16::INTEGER::BIT(12)) AS bit12,
    (16::INTEGER::BIT(12)) {{left_shift}} 3
FROM national
LIMIT 5;

Running query in 'postgresql://localhost:5432/gnis'

5 rows affected.

elev_in_m,bit12,?column?
931.0,000000010000,000010000000
2707.0,000000010000,000010000000
275.0,000000010000,000010000000
1685.0,000000010000,000010000000
1354.0,000000010000,000010000000


In [16]:
%%sql
EXPLAIN (verbose true)
WITH shifts AS (
    SELECT elev_in_m,
       (elev_in_m::integer::bit(12)) AS bit12,
       (elev_in_m::integer::bit(12) {{right_shift}} 8) AS rightshifted,
       ((elev_in_m::integer::bit(12) {{right_shift}} 8) {{left_shift}} 8)::integer AS round_to_256,
       ((elev_in_m::integer::bit(12) {{right_shift}} 8) {{left_shift}} 8)::integer % 256 AS test
  FROM national
)
SELECT COUNT(DISTINCT elev_in_m) AS elevation_meters_count,
       COUNT(DISTINCT bit12) AS bit12_count,
       COUNT(DISTINCT rightshifted) AS rightshift_count,
       COUNT(DISTINCT round_to_256) AS rounded_count
  FROM shifts;

Running query in 'postgresql://localhost:5432/gnis'

4 rows affected.

QUERY PLAN
Aggregate (cost=799.99..800.00 rows=1 width=32)
"Output: count(DISTINCT ""national"".elev_in_m), count(DISTINCT ((""national"".elev_in_m)::integer)::bit(12)), count(DISTINCT (((""national"".elev_in_m)::integer)::bit(12) >> 8)), count(DISTINCT (((((""national"".elev_in_m)::integer)::bit(12) >> 8) << 8))::integer)"
"-> Seq Scan on public.""national"" (cost=0.00..396.33 rows=11533 width=8)"
"Output: ""national"".feature_id, ""national"".feature_name, ""national"".feature_class, ""national"".state_alpha, ""national"".state_numeric, ""national"".county_name, ""national"".county_numeric, ""national"".primary_lat_dms, ""national"".prim_long_dms, ""national"".prim_lat_dec, ""national"".prim_long_dec, ""national"".source_lat_dms, ""national"".source_long_dms, ""national"".source_lat_dec, ""national"".source_long_dec, ""national"".elev_in_m, ""national"".elev_in_ft, ""national"".map_name, ""national"".date_created, ""national"".date_edited"


# [At Home] Demo 1: Roll-up / Drill-down Practice

Let's start with county-level data on elevations:

In [17]:
%%sql
SELECT state_numeric, county_numeric,
       avg(elev_in_m),
       stddev(elev_in_m), count(*)
FROM national TABLESAMPLE BERNOULLI(10)
GROUP BY state_numeric, county_numeric;

Running query in 'postgresql://localhost:5432/gnis'

852 rows affected.

state_numeric,county_numeric,avg,stddev,count
56,35.0,2804.0,None,1
39,51.0,235.0,None,1
53,37.0,557.0,None,1
13,37.0,66.0,None,1
53,73.0,169.0,None,1
36,119.0,50.0,None,1
36,65.0,326.0,200.8183258569795,2
45,45.0,258.0,None,1
47,95.0,88.0,None,2
51,540.0,194.0,None,1


**Roll up** to state level.
* We save the view as `state_elevations` for later...

In [37]:
%%sql
DROP VIEW IF EXISTS state_elevations;

CREATE VIEW state_elevations AS (
    SELECT state_numeric,
       avg(elev_in_m),
       stddev(elev_in_m), count(*)
    FROM national
    GROUP BY state_numeric
);

Running query in 'postgresql://localhost:5432/gnis'

++
||
++
++

In [38]:
%sql SELECT * FROM state_elevations;

Running query in 'postgresql://localhost:5432/gnis'

59 rows affected.

state_numeric,avg,stddev,count
54,363.6190476190476,199.26650831834746,204
29,246.09152542372883,80.2483078596168,343
68,6.666666666666667,7.99166232186187,14
4,1315.3798076923076,672.6305522946129,208
34,40.08943089430894,59.88896941733248,123
51,254.55197132616487,260.54513270095333,283
70,18.333333333333332,31.75426480542942,3
10,22.11111111111111,28.015563440198648,27
35,1756.8467432950192,471.8002505531821,273
45,122.83240223463687,123.96059930539184,181


**Drill down** to include feature class.

In [20]:
%%sql
SELECT state_numeric, feature_class,
       avg(elev_in_m),
       stddev(elev_in_m), count(*)
FROM national TABLESAMPLE Bernoulli(10)
GROUP BY state_numeric, feature_class
ORDER BY count(*) DESC;

Running query in 'postgresql://localhost:5432/gnis'

567 rows affected.

state_numeric,feature_class,avg,stddev,count
48,Church,281.1666666666667,266.01156274709473,12
48,Stream,306.54545454545456,252.4117127378853,11
30,Well,972.7272727272727,172.8658965262327,11
6,School,90.9090909090909,104.60444975760309,11
47,Cemetery,272.7,93.76927949908637,10
17,School,189.0,17.663521732655695,9
35,Locale,1725.5714285714287,505.5900371984061,8
42,Building,228.625,83.39053988826996,8
47,Stream,246.75,107.29365578368821,8
29,Cemetery,268.0,62.39505474909736,8


# Demo 2: Connections to Statistics

## [At home] Roll up with marginal distributions

In [8]:
%%sql
SELECT state_numeric,
       AVG(elev_in_m),
       STDDEV(elev_in_m), COUNT(*),
       SUM(COUNT(*)) OVER () AS total,
       COUNT(*)/SUM(COUNT(*)) OVER () AS marginal
FROM national TABLESAMPLE Bernoulli(.07)
GROUP BY state_numeric;

Running query in 'postgresql://localhost:5432/gnis'

7 rows affected.

state_numeric,avg,stddev,count,total,marginal
2,114.0,None,1,7,0.14285714285714285714
6,242.0,None,1,7,0.14285714285714285714
16,2541.0,None,1,7,0.14285714285714285714
23,43.0,None,1,7,0.14285714285714285714
26,178.0,None,1,7,0.14285714285714285714
40,579.0,None,1,7,0.14285714285714285714
45,259.0,None,1,7,0.14285714285714285714


In [9]:
%%sql
SELECT COUNT(DISTINCT county_numeric) FROM national;

Running query in 'postgresql://localhost:5432/gnis'

1 rows affected.

count
291


## Drill down with normally-distributed elevations:

We will start with a view that is at the state granularity, and then go from there. (Same as "At Home" exercise above)

In [41]:
%%sql
DROP VIEW IF EXISTS state_elevations;

CREATE VIEW state_elevations AS (
    SELECT state_numeric,
       avg(elev_in_m),
       stddev(elev_in_m), count(*)
    FROM national
    GROUP BY state_numeric
);

Running query in 'postgresql://localhost:5432/gnis'

++
||
++
++

Start with the `state_elevations` view from earlier:

In [42]:
%sql SELECT * FROM state_elevations;

Running query in 'postgresql://localhost:5432/gnis'

59 rows affected.

state_numeric,avg,stddev,count
54,363.6190476190476,199.26650831834746,204
29,246.09152542372883,80.2483078596168,343
68,6.666666666666667,7.99166232186187,14
4,1315.3798076923076,672.6305522946129,208
34,40.08943089430894,59.88896941733248,123
51,254.55197132616487,260.54513270095333,283
70,18.333333333333332,31.75426480542942,3
10,22.11111111111111,28.015563440198648,27
35,1756.8467432950192,471.8002505531821,273
45,122.83240223463687,123.96059930539184,181


The `fips_counties` relation has all counties, including those not in `national`:

In [13]:
%sql SELECT * FROM fips_counties LIMIT 10;

Running query in 'postgresql://localhost:5432/gnis'

10 rows affected.

fips,county,state_numeric
1000,Alabama,1
1001,Autauga County,1
1003,Baldwin County,1
1005,Barbour County,1
1007,Bibb County,1
1009,Blount County,1
1011,Bullock County,1
1013,Butler County,1
1015,Calhoun County,1
1017,Chambers County,1


If we wanted to **drill down** to the FIPS counties, we'd need to simulate an elevation for those counties that don't exist in `national`.

Here's the first step in that process, which creates a simulated value for *every* county in `fips_counties`.
* The value is simulated from a normal distribution using that state's elevation statistics (average, standard deviation).
* Just like a Python package, we need to import `tablefunc` in order to use the `normal_rand` function.

A few other details:
* Need `WITH ORDINALITY` to add row numbers to each sample generated via `normal_rand`
* We join on these row numbers to ensure that each county gets a unique sampled value

In [43]:
%sql CREATE EXTENSION IF NOT EXISTS tablefunc;

Running query in 'postgresql://localhost:5432/gnis'

++
||
++
++

In [44]:
%%sql
WITH state_cty AS
(SELECT s.state_numeric, f.fips as county_numeric, s.avg, s.stddev, s.count, rank() OVER (PARTITION BY s.state_numeric ORDER BY f.fips) as rk
  FROM state_elevations s, fips_counties f
  WHERE s.state_numeric = f.state_numeric
)
SELECT s.*,
       n.rn AS sampled_rk,
       n.n AS elev_in_m,
       true as elev_in_m_sim -- user-facing flag
  FROM state_cty s,
       LATERAL (SELECT n, rn FROM normal_rand(CAST(s.count AS INTEGER), s.avg, s.stddev) WITH ORDINALITY AS n(n, rn)) n
  WHERE s.rk = n.rn
LIMIT 10;

Running query in 'postgresql://localhost:5432/gnis'

10 rows affected.

state_numeric,county_numeric,avg,stddev,count,rk,sampled_rk,elev_in_m,elev_in_m_sim
1,1000,146.37888198757764,102.92185851771194,339,1,1,56.90046019750608,True
1,1001,146.37888198757764,102.92185851771194,339,2,2,213.06634933566627,True
1,1003,146.37888198757764,102.92185851771194,339,3,3,23.284003055592848,True
1,1005,146.37888198757764,102.92185851771194,339,4,4,151.9617337154284,True
1,1007,146.37888198757764,102.92185851771194,339,5,5,329.96425880353996,True
1,1009,146.37888198757764,102.92185851771194,339,6,6,160.99681714734837,True
1,1011,146.37888198757764,102.92185851771194,339,7,7,29.734467336642723,True
1,1013,146.37888198757764,102.92185851771194,339,8,8,128.32453691234738,True
1,1015,146.37888198757764,102.92185851771194,339,9,9,222.8307966482357,True
1,1017,146.37888198757764,102.92185851771194,339,10,10,153.7432441835299,True


# Assembling an Explicit Hierarchy

In [45]:
## we'll use the Lahman baseball database in our initial examples today.
## replace the database connection with a database of your own!
%reload_ext sql
%sql postgresql://localhost:5432/baseball

Switching to connection 'postgresql://localhost:5432/baseball'

Two relations have the pieces of the hierarchy we want:

In [46]:
%sql SELECT * FROM Appearances WHERE year_id > 1970 LIMIT 2;

Running query in 'postgresql://localhost:5432/baseball'

2 rows affected.

year_id,team_id,lg_id,player_id,g_all,gs,g_batting,g_defense,g_p,g_c,g_1b,g_2b,g_3b,g_ss,g_lf,g_cf,g_rf,g_of,g_dh,g_ph,g_pr
1971,ATL,NL,aaronha01,139,129,139,129,0,0,71,0,0,0,0,0,60,60,0,10,0
1971,ATL,NL,aaronto01,25,10,25,18,0,0,11,0,7,0,0,0,0,0,0,8,0


In [47]:
%sql SELECT * FROM Teams LIMIT 1;

Running query in 'postgresql://localhost:5432/baseball'

1 rows affected.

year_id,lg_id,team_id,franch_id,div_id,rank,g,ghome,w,l,divwin,wcwin,lgwin,wswin,r,ab,h,h2b,h3b,hr,bb,so,sb,cs,hbp,sf,ra,er,era,cg,sho,sv,ipouts,ha,hra,bba,soa,e,dp,fp,name,park,attendance,bpf,ppf,team_idbr,team_idlahman45,team_idretro
1871,NA,BS1,BNA,None,3,31,None,20,10,None,None,N,None,401,1372,426,70,37,3,60,19,73,16,None,None,303,109,3.55,22,1,3,828,367,2,42,23,243,24,0.834,Boston Red Stockings,South End Grounds I,None,103,98,BOS,BS1,BS1


Let's join these two to make our hierarchy! Which way should we make this?

In [48]:
%%sql
SELECT a.player_id, a.team_id, t.div_id, a.*
FROM Appearances a
NATURAL JOIN Teams t
WHERE a.year_id = 2015
LIMIT 100;

Running query in 'postgresql://localhost:5432/baseball'

100 rows affected.

player_id,team_id,div_id,year_id,team_id_1,lg_id,player_id_1,g_all,gs,g_batting,g_defense,g_p,g_c,g_1b,g_2b,g_3b,g_ss,g_lf,g_cf,g_rf,g_of,g_dh,g_ph,g_pr
alvarda02,BAL,E,2015,BAL,AL,alvarda02,12,10,12,12,0,0,0,0,0,0,0,1,12,12,0,0,0
brachbr01,BAL,E,2015,BAL,AL,brachbr01,62,0,5,62,62,0,0,0,0,0,0,0,0,0,0,0,0
brittza01,BAL,E,2015,BAL,AL,brittza01,64,0,2,64,64,0,0,0,0,0,0,0,0,0,0,0,0
cabrace01,BAL,E,2015,BAL,AL,cabrace01,2,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0
cabreev01,BAL,E,2015,BAL,AL,cabreev01,29,28,29,28,0,0,0,2,0,27,0,0,0,0,0,0,1
chenwe02,BAL,E,2015,BAL,AL,chenwe02,31,31,0,31,31,0,0,0,0,0,0,0,0,0,0,0,0
clevest01,BAL,E,2015,BAL,AL,clevest01,30,24,30,10,0,9,1,0,0,0,0,0,0,0,18,4,0
davisch02,BAL,E,2015,BAL,AL,davisch02,160,159,160,138,0,0,111,0,0,0,0,0,30,30,22,0,0
deazaal01,BAL,E,2015,BAL,AL,deazaal01,30,27,30,27,0,0,0,0,0,0,19,0,13,27,0,3,0
drakeol01,BAL,E,2015,BAL,AL,drakeol01,13,0,1,13,13,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
%%sql
CREATE OR REPLACE VIEW bball_tree AS (
    SELECT DISTINCT
        a.player_id, a.team_id, t.div_id,
        a.lg_id, a.year_id
    FROM appearances a
    NATURAL JOIN teams t
);

Running query in 'postgresql://localhost:5432/baseball'

++
||
++
++

In [50]:
%sql SELECT * FROM bball_tree LIMIT 25;

Running query in 'postgresql://localhost:5432/baseball'

25 rows affected.

player_id,team_id,div_id,lg_id,year_id
gumbeha01,NY1,None,NL,1935
gradymi01,SLN,None,NL,1897
deshoji01,WS1,None,AL,1938
prattla01,BRF,None,FL,1915
thompsa01,PHI,None,NL,1890
hollica01,DET,None,AL,1922
halege01,SLA,None,AL,1916
mamaual01,NYA,None,AL,1924
henryji01,BOS,None,AL,1937
cristch01,PHI,None,NL,1906


### Revisiting the Home Run Query

Recall our old home run query:

In [51]:
%%sql
SELECT name_first, name_last, year_id,
       MIN(hr), MAX(hr), AVG(hr), STDDEV(hr), SUM(hr)
FROM batting b, people p
WHERE b.player_id = p.player_id
GROUP BY name_last, name_first, year_id
ORDER BY max DESC
LIMIT 10;

Running query in 'postgresql://localhost:5432/baseball'

10 rows affected.

name_first,name_last,year_id,min,max,avg,stddev,sum
Barry,Bonds,2001,73,73,73.0000000000000000,None,73
Mark,McGwire,1998,70,70,70.0000000000000000,None,70
Sammy,Sosa,1998,66,66,66.0000000000000000,None,66
Mark,McGwire,1999,65,65,65.0000000000000000,None,65
Sammy,Sosa,2001,64,64,64.0000000000000000,None,64
Sammy,Sosa,1999,63,63,63.0000000000000000,None,63
Roger,Maris,1961,61,61,61.0000000000000000,None,61
Babe,Ruth,1927,60,60,60.0000000000000000,None,60
Babe,Ruth,1921,59,59,59.0000000000000000,None,59
Giancarlo,Stanton,2017,59,59,59.0000000000000000,None,59


Set up for roll up/drill down on `bball_tree` hierarchy.
* Join each (raw) person with the associated `bball_tree` entry by `(playerid, yearid)` in a CTE
* Use this result for roll-up and drill-down.

(blank space before we get to the next exercise....)
<br/><br/><br/><br/><br/>
<br/><br/><br/><br/><br/>
<br/><br/><br/><br/><br/>
<br/><br/><br/><br/><br/>

In [53]:
%%sql
WITH batting_tree AS (
    SELECT b.*, t.div_id
    FROM batting b, bball_tree t
    WHERE b.player_id = t.player_id
      AND b.year_id = t.year_id
)
SELECT name_first, name_last,
       bt.team_id, bt.lg_id, bt.div_id, bt.year_id,
       MIN(hr), MAX(hr), AVG(hr), STDDEV(hr), SUM(hr)
FROM batting_tree bt, people p
WHERE bt.player_id = p.player_id
GROUP BY bt.player_id, bt.team_id, bt.lg_id, bt.div_id, bt.year_id, name_last, name_first
ORDER BY max DESC
LIMIT 10;


Running query in 'postgresql://localhost:5432/baseball'

10 rows affected.

name_first,name_last,team_id,lg_id,div_id,year_id,min,max,avg,stddev,sum
Barry,Bonds,SFN,NL,W,2001,73,73,73.0000000000000000,None,73
Mark,McGwire,SLN,NL,C,1998,70,70,70.0000000000000000,None,70
Sammy,Sosa,CHN,NL,C,1998,66,66,66.0000000000000000,None,66
Mark,McGwire,SLN,NL,C,1999,65,65,65.0000000000000000,None,65
Sammy,Sosa,CHN,NL,C,2001,64,64,64.0000000000000000,None,64
Sammy,Sosa,CHN,NL,C,1999,63,63,63.0000000000000000,None,63
Roger,Maris,NYA,AL,None,1961,61,61,61.0000000000000000,None,61
Babe,Ruth,NYA,AL,None,1927,60,60,60.0000000000000000,None,60
Babe,Ruth,NYA,AL,None,1921,59,59,59.0000000000000000,None,59
Giancarlo,Stanton,MIA,NL,E,2017,59,59,59.0000000000000000,None,59
